Steps
* calculate size of electrode areas 4680 vs 280Ah vs 500Ah 
* determine number of windings / sheets 
* calculate number of current collector tabs 
* cell volume utilization ratio (electode-seperator-current_collector volume / cell can volume) --> why?
* resistance / impedance to calculate Joule Heating IR² (only valid for external short circuit, not internal)


In [2]:
import sys
sys.path.insert(0, '/home/matthias/Documents/GitHub/Cell_model_energy_costs')
import cell_model as Cellmodel

## Making the cells

In [99]:
# Active materials
NMC_cathode = Cellmodel.Activematerial_cathode("NMC811" , 195, 3.86)
LFP_cathode = Cellmodel.Activematerial_cathode("LFP", 160, 3.3)
Gr = Cellmodel.Activematerial_anode("Gr", 344 , 0.17)

# Currentcollectors 
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)

# Electrolyte 
LP40_standard = Cellmodel.Electrolyte_liquid("LiPF6" , "EC:DMC",
                                      1.1, # concentration mol / L
                                      151.9 # MolarMass LiPF6
                                      )

# Separator
Separator_standard = Cellmodel.Separator("PP+Al" , 12e-4 , 0.44 , 1.18)

# Electrods 
positive_NMC = Cellmodel.Electrodecomposition_cathode_opt1(NMC_cathode.name,
                                                       3.3, # areal capacity
                                                       0.95, # active frac
                                                       3.4, # density
                                                       NMC_cathode
                                                      ) # NMC = 3.4 / LFP = 2.5
positive_LFP = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name, 
                                                        2.5, 
                                                        0.95, 
                                                        2.5, 
                                                        LFP_cathode
                                                        ) # NMC = 3.4 / LFP = 2.5
negative = Cellmodel.Electrodecomposition_anode_opt1( Gr.name , 3.3*1.1 , 0.965 , 1.6, Gr)

### Cylindrical

In [22]:
NMC_Gr_cyl = Cellmodel.Cylindrical("NMC-Gr_Cyl", 
                               positive_NMC, 
                               negative, 
                               Separator_standard , 
                               LP40_standard,
                               1.7, # electrolyte content mL/mAh
                               4.6, # diameter
                               8.0, # height 
                               0.0165, # can thickness
                               7.9, # can density
                               0.15, # void diameter
                               0.4, # head space
                               4, # extra mass
                               0.94, # LLI
                               NMC_cathode, 
                               Gr, 
                               Al, 
                               Cu)

In [23]:
NMC_Gr_cyl.jr_area

4199.490203027328

In [24]:
NMC_Gr_cyl.turns

75.82406919653891

In [25]:
positive_NMC.thickness_el_c

0.00523934270064301

In [26]:
NMC_Gr_cyl.energy

96.13792134025589

In [27]:
NMC_Gr_cyl.capacity

26.053637219581542

Seems to fit well to the Tesla 4680 NMC811 cell https://insideevs.com/news/598656/tesla-4680-battery-cell-specs/ 

### Prismatic

In [119]:
LFP_Gr_prismatic = Cellmodel.Prismatic("LFP-Gr_Pris", 
                                       positive_LFP, 
                                       negative, 
                                       Separator_standard, 
                                       LP40_standard, 
                                       1.7, # electrolyte ratio mL / mAh
                                       17.2, # height
                                       20.6, # width 
                                       7.2, # depth
                                       0.01, # can thickness
                                       7.9, # can density
                                       1, # terminal clearance
                                       18, # nrolls
                                       40, # extramass
                                       0.94, # LLI
                                       LFP_cathode, 
                                       Gr, 
                                       Al, 
                                       Cu)

In [120]:
LFP_Gr_prismatic.jr_area

59879.53990627049

In [121]:
LFP_Gr_prismatic.turns

5.797544652112828

In [122]:
LFP_Gr_prismatic.energy

880.8879115611451

In [123]:
LFP_Gr_prismatic.capacity

281.43383755947127

Seems to fit to the CATL 280 Ah cell https://www.evlithium.com/CATL-Battery-Cell/catl-lifepo4-battery-280ah.html 